## Import necessary libraries

In [32]:
import xml.etree.ElementTree as ET
import os
from os import listdir
from os.path import join

## Define classes

In [33]:
classes = ["mask_weared_incorrect", "with_mask", "without_mask"]

## Normalizing coordinates

In [34]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

## Extracting coordinates from .xml files

In [35]:
def convert_annotation(image_id):
    in_file = open('annotations/%s.xml'%(image_id), encoding = 'utf-8')
    out_file = open('labels/%s.txt'%(image_id), 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
 
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

## Saving data

In [36]:
val_percent = 0.1
data_path = '../dataset/images/'
if not os.path.exists('labels/'):
    os.makedirs('labels/')
image_ids = [f for f in os.listdir ( 'annotations')] # XML data storage folder
train_file = open('train.txt', 'w')
val_file = open('test.txt', 'w')
for i, image_id in enumerate(image_ids):
    if image_id [-3:] == "xml": # Sometimes jpg and xml files are placed in the same folder, so to determine what suffixes
        if i < (len(image_ids) * val_percent):
            val_file.write(data_path + '%s\n'%(image_id[:-3] + 'jpg'))
        else:
            train_file.write(data_path + '%s\n'%(image_id[:-3] + 'jpg'))
    convert_annotation(image_id[:-4])
train_file.close()
val_file.close()